In [1]:
from ipywidgets import interact
import tensorflow as tf
session = tf.InteractiveSession()

In [2]:
from data import decode_output_sequences
from model import Seq2SeqModel

# Date normalization model

In [9]:
from date_generator import generate_data, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN

session.close()
tf.reset_default_graph()
session = tf.InteractiveSession()

hidden_units = 128
batch_size = 32
num_symbols = len(SYMBOL_TO_IDX)

with tf.variable_scope('model', reuse=None):
    training_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size = batch_size,
                                    is_training=True)

training_model.init_variables()

with tf.variable_scope('model', reuse=True):
    testing_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size = batch_size,
                                    is_training=False)

print("Finished building model")

Finished building model


Generate some data to look at.

In [23]:
from date_generator import generate_data, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN 

date_generator = generate_data(batch_size=10, random_format=False)
x, y = next(date_generator)

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

pprint([("Input", "Target")] + 
       list(zip(input_strings, target_strings)))

[('Input', 'Target'),
 ('____1102 ,21 tsuguaG', '2011-08-12___'),
 ('__91 ,rebmeced 1102G', '2011-12-19___'),
 ('___________82-01-21G', '2028-10-12___'),
 ('___________17-21-72G', '1971-12-27___'),
 ('_____2002 ,90 lirpaG', '2002-04-09___'),
 ('___1991 ,31 rebotcoG', '1991-10-13___'),
 ('___________13-70-07G', '1970-07-31___'),
 ('_________10-60-2202G', '2022-06-01___'),
 ('_70 ,rebmetpes 3591G', '1953-09-07___'),
 ('___________32-20-04G', '2040-02-23___')]


In [17]:
date_generator = generate_data(batch_size=32)
training_model.fit(date_generator, num_epochs=20, batches_per_epoch=64)
print("Finished training")

63/64 (epoch 0), train_loss = 0.151, time/batch = 0.092
63/64 (epoch 1), train_loss = 0.157, time/batch = 0.081
63/64 (epoch 2), train_loss = 0.189, time/batch = 0.067
63/64 (epoch 3), train_loss = 0.132, time/batch = 0.107
63/64 (epoch 4), train_loss = 0.137, time/batch = 0.068
63/64 (epoch 5), train_loss = 0.113, time/batch = 0.062
63/64 (epoch 6), train_loss = 0.145, time/batch = 0.067
63/64 (epoch 7), train_loss = 0.131, time/batch = 0.105
63/64 (epoch 8), train_loss = 0.124, time/batch = 0.077
63/64 (epoch 9), train_loss = 0.092, time/batch = 0.066
63/64 (epoch 10), train_loss = 0.098, time/batch = 0.068
63/64 (epoch 11), train_loss = 0.126, time/batch = 0.068
63/64 (epoch 12), train_loss = 0.107, time/batch = 0.099
63/64 (epoch 13), train_loss = 0.099, time/batch = 0.071
63/64 (epoch 14), train_loss = 0.120, time/batch = 0.066
63/64 (epoch 15), train_loss = 0.093, time/batch = 0.067
63/64 (epoch 16), train_loss = 0.093, time/batch = 0.064
63/64 (epoch 17), train_loss = 0.079, tim

In [18]:
from pprint import pprint

batch_size = 20
date_generator = generate_data(batch_size=batch_size, random_format=False)

x, y = next(date_generator)
input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

model_output = testing_model.predict(x)
pred_strings = decode_output_sequences(model_output, symbols=SYMBOLS)

pprint([("Input", "Target", "Output")] + 
       list(zip(input_strings, target_strings, pred_strings)))


[('Input', 'Target', 'Output'),
 ('_________51/40/9991G', '1999-04-15___', '1999-04-15____'),
 ('_________11/01/3791G', '1973-10-11___', '1973-11-01____'),
 ('_______0591 ,60 yamG', '1950-05-06___', '1950-05-06____'),
 ('__________09 21 rpaG', '1990-04-12___', '2009-04-19____'),
 ('__________24 32 nujG', '2042-06-23___', '2024-06-23____'),
 ('___________82-50-15G', '1951-05-28___', '1951-05-28____'),
 ('__3591 ,32 yraurbefG', '1953-02-23___', '1953-02-23____'),
 ('_________02-80-7891G', '1987-08-20___', '1987-08-20____'),
 ('___________63-20-60G', '2036-02-06___', '2036-02-06____'),
 ('_______3691 ,50 yamG', '1963-05-05___', '1963-05-05____'),
 ('___________91-50-86G', '1968-05-19___', '1968-05-19____'),
 ('__________25 52 guaG', '1952-08-25___', '2025-08-25____'),
 ('___________48-40-62G', '1984-04-26___', '1984-04-26____'),
 ('___________04-40-31G', '2040-04-13___', '2040-04-13____'),
 ('___7791 ,10 yraunajG', '1977-01-01___', '1977-01-01____'),
 ('__________80 92 guaG', '2008-08-29

# Addition model

Train the model to add two numbers, x+y, and output the sum.

Generate some example data:

In [3]:
from addition_generator import AdditionGenerator, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN 

addition_generator = AdditionGenerator(batch_size=3)
x, y = addition_generator.next_batch()

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

print(" Inputs:", input_strings)
print("Targets:", target_strings)

 Inputs: ['________92+4G', '________73+0G', '_________9+6G']
Targets: ['96_____', '73_____', '15_____']


In [50]:
session.close()
tf.reset_default_graph()
session = tf.InteractiveSession()

hidden_units = 128
num_layers = 2
training_batch_size = 32
num_symbols = len(SYMBOL_TO_IDX)

with tf.variable_scope('model', reuse=None):
    training_model = Seq2SeqModel(session=session,
                                    hidden_units=hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=True)

training_model.init_variables()

with tf.variable_scope('model', reuse=True):
    testing_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=False)

addition_generator = AdditionGenerator(batch_size=training_batch_size)

print("Finished building model")

Finished building model


In [53]:
training_model.fit_curr(addition_generator, testing_model=testing_model, num_epochs=40, batches_per_epoch=20)
print("Finished training")

Epoch 0: train_loss = 0.012, val_loss = 0.111, time/epoch = 1.580
Epoch 1: train_loss = 0.047, val_loss = 0.142, time/epoch = 1.965
Epoch 2: train_loss = 0.024, val_loss = 0.099, time/epoch = 1.821
Epoch 3: train_loss = 0.018, val_loss = 0.092, time/epoch = 1.784
Epoch 4: train_loss = 0.016, val_loss = 0.090, time/epoch = 1.783
Epoch 5: train_loss = 0.024, val_loss = 0.076, time/epoch = 1.648
Epoch 6: train_loss = 0.015, val_loss = 0.074, time/epoch = 1.716
Epoch 7: train_loss = 0.031, val_loss = 0.048, time/epoch = 1.725
Increasing difficulty
Epoch 8: train_loss = 0.627, val_loss = 0.806, time/epoch = 1.739
Epoch 9: train_loss = 0.523, val_loss = 0.699, time/epoch = 1.749
Epoch 10: train_loss = 0.459, val_loss = 0.634, time/epoch = 1.821
Epoch 11: train_loss = 0.378, val_loss = 0.595, time/epoch = 1.814
Epoch 12: train_loss = 0.434, val_loss = 0.587, time/epoch = 1.980
Epoch 13: train_loss = 0.315, val_loss = 0.569, time/epoch = 1.734
Epoch 14: train_loss = 0.364, val_loss = 0.583, ti

In [56]:
from pprint import pprint

batch_size = 10
test_generator = AdditionGenerator(batch_size=batch_size, number_len=3)

x, y = test_generator.next_batch()
input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

model_output = testing_model.predict(x)
pred_strings = decode_output_sequences(model_output, symbols=SYMBOLS)

print("Error rate:", testing_model.validate([(x, y)]))

pprint([("Input", "Target", "Output")] + 
       list(zip(input_strings, target_strings, pred_strings)))


Error rate: 0.4
[('Input', 'Target', 'Output'),
 ('_____631+962G', '1593___', '1663____'),
 ('________0+10G', '10_____', '10______'),
 ('________0+80G', '80_____', '80______'),
 ('______142+10G', '152____', '152_____'),
 ('_______271+1G', '272____', '282_____'),
 ('________4+38G', '42_____', '42______'),
 ('______824+10G', '834____', '824_____'),
 ('_____843+365G', '1208___', '1228____'),
 ('________18+1G', '19_____', '19______'),
 ('_______16+28G', '44_____', '44______')]


# Program execution model

In [58]:
from program_generator import ProgramGenerator, SYMBOLS, SYMBOL_TO_IDX, INPUT_SEQ_LEN, OUTPUT_SEQ_LEN 

program_generator = ProgramGenerator(batch_size=10, program_length=1, num_len=2)
x, y = program_generator.next_batch()

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

print(" Inputs:", input_strings)
print("Targets:", target_strings)

 Inputs: ['________________________________________________________________________a=3\nif a>9:\n  a+=3\nprint(a)G', '_____________________________________________________________________h=17\nif h>85:\n  h=h-1\nprint(h)G', '________________________________________________________________________________h=25\nd=1\nprint(h+d)G', '_________________________________________________________________________________d=9\nd-=11\nprint(d)G', '_______________________________________________________________________________f=72\nd=48\nprint(f+d)G', '________________________________________________________________________________e=70\ne=e-3\nprint(e)G', '________________________________________________________________________________d=1\nd=d+93\nprint(d)G', '_________________________________________________________________________________h=0\nf=7\nprint(h-f)G', '________________________________________________________________________________b=5\nh=24\nprint(h+b)G', '_________________________________

In [59]:
session.close()
tf.reset_default_graph()
session = tf.InteractiveSession()

hidden_units = 160
num_layers = 2
training_batch_size = 32
num_symbols = len(SYMBOL_TO_IDX)

with tf.variable_scope('model', reuse=None):
    training_model = Seq2SeqModel(session=session,
                                    hidden_units=hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=True)

training_model.init_variables()

with tf.variable_scope('model', reuse=True):
    testing_model = Seq2SeqModel(session=session,
                                    hidden_units = hidden_units, 
                                    num_layers=num_layers,
                                    input_sequence_len = INPUT_SEQ_LEN,
                                    output_sequence_len = OUTPUT_SEQ_LEN,
                                    num_input_symbols = num_symbols,
                                    num_output_symbols = num_symbols,
                                    batch_size=training_batch_size,
                                    is_training=False)

program_generator = ProgramGenerator(batch_size=training_batch_size, program_length=1, num_len=2)

print("Finished building model")

Finished building model


In [60]:
training_model.fit_curr(program_generator, 
                        testing_model=testing_model, 
                        num_epochs=20000, 
                        batches_per_epoch=128)
print("Finished training")

Epoch 0: train_loss = 0.536, val_error_rate = 0.994, time/epoch = 84.344
Epoch 1: train_loss = 0.517, val_error_rate = 0.990, time/epoch = 77.085
Epoch 2: train_loss = 0.540, val_error_rate = 0.974, time/epoch = 77.576
Epoch 3: train_loss = 0.530, val_error_rate = 0.989, time/epoch = 74.290
Epoch 4: train_loss = 0.498, val_error_rate = 0.971, time/epoch = 76.031
Epoch 5: train_loss = 0.487, val_error_rate = 0.970, time/epoch = 74.731
Epoch 6: train_loss = 0.474, val_error_rate = 0.967, time/epoch = 74.710
Epoch 7: train_loss = 0.451, val_error_rate = 0.970, time/epoch = 76.828
Epoch 8: train_loss = 0.422, val_error_rate = 0.968, time/epoch = 81.042
Epoch 9: train_loss = 0.403, val_error_rate = 0.950, time/epoch = 77.604
Epoch 10: train_loss = 0.370, val_error_rate = 0.943, time/epoch = 81.424
Epoch 11: train_loss = 0.338, val_error_rate = 0.937, time/epoch = 78.480
Epoch 12: train_loss = 0.322, val_error_rate = 0.906, time/epoch = 78.038
Epoch 13: train_loss = 0.323, val_error_rate = 0

KeyboardInterrupt: 

In [70]:
batch_size = 100
test_generator = ProgramGenerator(batch_size=batch_size, num_len=2, program_length=1)

x, y = test_generator.next_batch()

input_strings = decode_output_sequences(x, symbols=SYMBOLS)
target_strings = decode_output_sequences(y, symbols=SYMBOLS)

model_output = testing_model.predict(x)

pred_strings = decode_output_sequences(model_output, symbols=SYMBOLS)

def view_prediction(i):
    print(input_strings[i].strip('_').strip('G'))
    print("-----")
    print("Targ:", target_strings[i].strip('_'))
    print("Pred:", pred_strings[i].strip('_'))
    print(y.shape, model_output.shape)
a = interact(view_prediction, i=(0, batch_size - 1))

f=18
if f>8:
  f-=54
print(f)
-----
Targ: -36
Pred: -46
(100, 10, 48) (100, 11, 48)


In [69]:
#%%prun
#from model import validate_model
testing_model.validate([test_generator.next_batch() for _ in range(30)])

0.09666666666666668